In [44]:
# =============================================================================
# DUSKPROBE v4.5 - WEB VULNERABILITY SCANNER
# =============================================================================
# This is an enhanced version for Google Colab with:
# - More detailed reporting with pandas DataFrame
# - Export options for reports and logs
# - Additional security checks (Open Redirect, IDOR, CSRF)
# - Improved performance and timing
# - Better error handling and logging
# - Professional report formatting

In [45]:
# =============================================================================
# STEP 1: INSTALL DEPENDENCIES (Automatic)
# =============================================================================
!pip install -q requests beautifulsoup4 colorama fake-useragent urllib3
!pip install -q aiohttp cryptography scikit-learn pybloom-live pandas numpy
!apt-get update -qq && apt-get install -y -qq tor

print("✅ Dependencies installed! Ready for scanning...")

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
✅ Dependencies installed! Ready for scanning...


In [46]:
# =============================================================================
# COMPLETE CODE
# =============================================================================
"""
DuskProbe v4.5 - Professional Web Vulnerability Scanner
Optimized for Google Colab: Fast, lightweight, interactive URL input.
Performs security checks for XSS, SQLi, LFI, security headers, cryptominers,
Open Redirect, IDOR, and CSRF.
"""

'\nDuskProbe v4.5 - Professional Web Vulnerability Scanner\nOptimized for Google Colab: Fast, lightweight, interactive URL input.\nPerforms security checks for XSS, SQLi, LFI, security headers, cryptominers,\nOpen Redirect, IDOR, and CSRF.\n'

In [47]:
# Standard library imports
import os  # Interact with the operating system (e.g., file paths, environment variables)
import re  # Regular expressions for pattern matching and text manipulation
import json  # Encode and decode JSON data
import time  # Time-related functions (e.g., sleep, timestamps)
import socket  # Network communication (e.g., IP resolution, socket connections)
import base64  # Encode/decode data in Base64 format
import hashlib  # Generate cryptographic hashes (e.g., SHA256)
import subprocess  # Run external processes and shell commands
import logging  # Logging system for tracking events and debugging
from logging.handlers import RotatingFileHandler  # Log handler that rotates logs when they reach a certain size

# Third-party libraries
import requests  # HTTP requests library for interacting with web APIs
import pandas as pd  # Data manipulation and analysis using DataFrames
import numpy as np  # Numerical computations and array operations

# URL and path utilities
from urllib.parse import urlparse, urljoin, quote  # Parse and manipulate URLs
from pathlib import Path  # Object-oriented file system paths

# Date and time handling
from datetime import datetime  # Work with date and time objects

# Type hints for better code clarity and static analysis
from typing import List, Dict, Tuple  # Define expected data types for variables and function signatures

# Retry logic for HTTP requests
from urllib3.util.retry import Retry  # Configure retry behavior for failed HTTP requests
from requests.adapters import HTTPAdapter  # Attach retry logic to requests sessions


In [48]:
# Colab detection
try:
    from google.colab import output
    from IPython.display import display, HTML, Markdown
    IS_COLAB = True
except ImportError:
    IS_COLAB = False

In [49]:
# Optional imports with fallbacks
try:
    from bs4 import BeautifulSoup
    BS4_AVAILABLE = True
except:
    BS4_AVAILABLE = False

try:
    from colorama import Fore, Style, init
    init(autoreset=True)
    COLOR_AVAILABLE = True
except:
    COLOR_AVAILABLE = False
    class Fore:
        RED, GREEN, YELLOW, BLUE, MAGENTA, CYAN, WHITE = "", "", "", "", "", "", ""
    class Style:
        BRIGHT, DIM, NORMAL = "", "", ""

try:
    from fake_useragent import UserAgent
    UA_AVAILABLE = True
except:
    UA_AVAILABLE = False

try:
    from stem.process import launch_tor_with_config
    from stem import Signal
    TOR_AVAILABLE = True
except:
    TOR_AVAILABLE = False

In [50]:
# Constants
MAX_PAGES = 5  # Optimized limit for Colab
REQUEST_TIMEOUT = 15
DEFAULT_USER_AGENT = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36"
REPORTS_DIR = "/content/reports"
LOGS_DIR = "/content/logs"
Path(REPORTS_DIR).mkdir(exist_ok=True)
Path(LOGS_DIR).mkdir(exist_ok=True)

In [51]:
# Setup logging
def setup_logging():
    """Setup comprehensive logging."""
    logger = logging.getLogger("DuskProbe")
    logger.setLevel(logging.INFO)

    # Remove existing handlers
    for handler in logger.handlers[:]:
        logger.removeHandler(handler)

    # File handler with rotation
    log_file = f"{LOGS_DIR}/duskprobe_{datetime.now().strftime('%Y%m%d_%H%M%S')}.log"
    file_handler = RotatingFileHandler(log_file, maxBytes=5*1024*1024, backupCount=3)
    file_handler.setLevel(logging.INFO)

    # Console handler
    console_handler = logging.StreamHandler()
    console_handler.setLevel(logging.INFO)

    # Formatter
    formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
    file_handler.setFormatter(formatter)
    console_handler.setFormatter(formatter)

    # Add handlers
    logger.addHandler(file_handler)
    logger.addHandler(console_handler)

    return logger, log_file

logger, log_file = setup_logging()
def print_colored(text, color=Fore.WHITE, style=Style.NORMAL):
    print(f"{style}{color}{text}{Style.RESET_ALL if COLOR_AVAILABLE else ''}")
    logger.info(text)


In [52]:
class SimpleSession:
    """Optimized lightweight session."""

    def __init__(self, use_tor=False):
        # Determine whether to use Tor based on availability
        self.use_tor = use_tor and TOR_AVAILABLE

        # Create a persistent HTTP session
        self.session = requests.Session()

        # Configure retry strategy for transient errors
        retry = Retry(total=2, backoff_factor=0.5, status_forcelist=[429, 500])
        adapter = HTTPAdapter(max_retries=retry)

        # Mount adapter to both HTTP and HTTPS protocols
        self.session.mount('http://', adapter)
        self.session.mount('https://', adapter)

        # Set default headers including User-Agent
        self.session.headers.update({
            'User-Agent': UserAgent().random if UA_AVAILABLE else DEFAULT_USER_AGENT,
            'Accept': 'text/html,application/xhtml+xml,*/*;q=0.9',
            'Connection': 'keep-alive'
        })

        # Initialize Tor if requested
        if use_tor:
            self._start_tor()

    def _start_tor(self):
        # Attempt to launch Tor with specified configuration
        try:
            self.tor_process = launch_tor_with_config(
                config={'SocksPort': '9050', 'ControlPort': '9051'}
            )

            # Set proxy settings to route traffic through Tor
            self.session.proxies = {
                'http': 'socks5h://127.0.0.1:9050',
                'https': 'socks5h://127.0.0.1:9050'
            }

            # Notify user that Tor is enabled
            print_colored("🛡️ Tor enabled", Fore.GREEN)
        except:
            # Fallback to direct connection if Tor fails
            print_colored("⚠️ Tor failed, using direct connection", Fore.YELLOW)
            self.use_tor = False

    def get(self, url: str) -> requests.Response:
        # Perform a GET request with retry on status 429
        try:
            resp = self.session.get(url, timeout=REQUEST_TIMEOUT, allow_redirects=True)
            if resp.status_code == 429:
                time.sleep(3)  # Wait before retrying
                resp = self.session.get(url, timeout=REQUEST_TIMEOUT)
            return resp
        except Exception as e:
            # Log and re-raise any request errors
            logger.error(f"Request error: {e}")
            raise

    def close(self):
        # Terminate Tor process if it was started
        if hasattr(self, 'tor_process') and self.tor_process:
            self.tor_process.terminate()


In [53]:

class SecurityChecker:
    """Core security checks - optimized for speed with additional tests."""
    def __init__(self, session):
        self.session = session
        self.payloads = {
            'xss': ["<script>alert(1)</script>", "javascript:alert(1)", "\"><script>alert(1)</script>"],
            'sqli': ["' OR 1=1--", "1' UNION SELECT 1--", "admin'--", "' OR 'a'='a"],
            'lfi': ["../../../etc/passwd", "/etc/passwd", "....//....//....//etc/passwd"],
            'redirect': ["https://evil.com", "//evil.com", "/\\evil.com"],
            'idor': ["/api/user/1", "/admin/1", "/profile/1"]
        }
        self.error_patterns = re.compile(r'(sql|mysql|ora|warning|error|syntax)', re.I)
        self.file_indicators = re.compile(r'(root:|bin|daemon|passwd|nologin)', re.I)
        self.miner_patterns = [
            re.compile(r'coin(hive|miner)', re.I), re.compile(r'cryptonight', re.I),
            re.compile(r'miner\.js', re.I), re.compile(r'xmrig', re.I),
            re.compile(r'webassembly\.instantiate', re.I)
        ]
        self.csrf_pattern = re.compile(r'<input[^>]+name=[\'"]csrf[\'"]', re.I)

    def get_site_info(self, url: str) -> Dict:
        """Get comprehensive site information."""
        info = {
            'url': url,
            'domain': '',
            'ip_address': '',
            'server': '',
            'technologies': [],
            'headers': {},
            'page_title': '',
            'forms_count': 0,
            'links_count': 0
        }

        try:
            resp = self.session.get(url)
            info['headers'] = dict(resp.headers)
            info['server'] = resp.headers.get('Server', 'Unknown')

            # Parse domain and IP
            parsed_url = urlparse(url)
            info['domain'] = parsed_url.netloc

            try:
                info['ip_address'] = socket.gethostbyname(parsed_url.netloc)
            except:
                info['ip_address'] = 'Unable to resolve'

            # Parse page content if available
            if BS4_AVAILABLE:
                soup = BeautifulSoup(resp.text, 'html.parser')
                info['page_title'] = soup.title.string if soup.title else 'No title'
                info['forms_count'] = len(soup.find_all('form'))
                info['links_count'] = len(soup.find_all('a'))

                # Detect technologies
                if 'X-Powered-By' in resp.headers:
                    info['technologies'].append(resp.headers['X-Powered-By'])
                if 'ASP.NET' in resp.headers.get('Set-Cookie', ''):
                    info['technologies'].append('ASP.NET')
                if 'wordpress' in resp.text.lower():
                    info['technologies'].append('WordPress')
                if 'jquery' in resp.text.lower():
                    info['technologies'].append('jQuery')

        except Exception as e:
            logger.warning(f"Site info collection failed: {e}")

        return info

    def check_xss(self, url: str) -> List[Dict]:
        findings = []
        for payload in self.payloads['xss']:
            test_url = f"{url}{'&' if '?' in url else '?'}q={quote(payload)}"
            try:
                resp = self.session.get(test_url)
                if payload.replace('<', '&lt;').replace('>', '&gt;') in resp.text or \
                   payload in resp.text:
                    evidence_location = self._find_evidence_location(resp.text, payload)
                    findings.append({
                        'type': 'XSS',
                        'severity': 'HIGH',
                        'details': f"Reflected XSS: {payload}",
                        'url': test_url,
                        'timestamp': datetime.now().isoformat(),
                        'evidence': f"Payload reflected in response. Location: {evidence_location}",
                        'risk_score': 8,
                        'affected_component': 'Input parameter',
                        'recommendation': 'Implement input validation and output encoding'
                    })
            except Exception as e:
                logger.warning(f"XSS check failed for {test_url}: {e}")
        return findings

    def check_sqli(self, url: str) -> List[Dict]:
        findings = []
        for payload in self.payloads['sqli']:
            test_url = f"{url}{'&' if '?' in url else '?'}id={quote(payload)}"
            try:
                resp = self.session.get(test_url)
                if self.error_patterns.search(resp.text):
                    error_match = self.error_patterns.search(resp.text)
                    evidence_location = self._find_evidence_location(resp.text, payload)
                    findings.append({
                        'type': 'SQLi',
                        'severity': 'CRITICAL',
                        'details': f"SQL Error: {payload}",
                        'url': test_url,
                        'timestamp': datetime.now().isoformat(),
                        'evidence': f"Database error detected: {error_match.group() if error_match else 'Unknown error'}. Location: {evidence_location}",
                        'risk_score': 9,
                        'affected_component': 'Database query',
                        'recommendation': 'Use parameterized queries and input validation'
                    })
            except Exception as e:
                logger.warning(f"SQLi check failed for {test_url}: {e}")
        return findings

    def check_lfi(self, url: str) -> List[Dict]:
        findings = []
        for payload in self.payloads['lfi']:
            test_url = f"{url}{'&' if '?' in url else '?'}file={quote(payload)}"
            try:
                resp = self.session.get(test_url)
                if self.file_indicators.search(resp.text):
                    file_match = self.file_indicators.search(resp.text)
                    evidence_location = self._find_evidence_location(resp.text, payload)
                    findings.append({
                        'type': 'LFI',
                        'severity': 'CRITICAL',
                        'details': f"File leak: {payload}",
                        'url': test_url,
                        'timestamp': datetime.now().isoformat(),
                        'evidence': f"Sensitive file content exposed: {file_match.group() if file_match else 'File content'}. Location: {evidence_location}",
                        'risk_score': 9,
                        'affected_component': 'File inclusion mechanism',
                        'recommendation': 'Validate file paths and implement access controls'
                    })
            except Exception as e:
                logger.warning(f"LFI check failed for {test_url}: {e}")
        return findings

    def check_headers(self, url: str) -> List[Dict]:
        findings = []
        try:
            resp = self.session.get(url)
            headers = {k.lower(): v for k, v in resp.headers.items()}

            # Check for missing security headers
            security_headers = {
                'content-security-policy': 'MEDIUM',
                'x-frame-options': 'MEDIUM',
                'strict-transport-security': 'HIGH',
                'x-content-type-options': 'LOW',
                'x-xss-protection': 'LOW'
            }

            for header, severity in security_headers.items():
                if header not in headers:
                    findings.append({
                        'type': 'Missing Header',
                        'severity': severity,
                        'details': f"Missing security header: {header}",
                        'url': url,
                        'timestamp': datetime.now().isoformat(),
                        'evidence': f"Header not present in HTTP response headers",
                        'risk_score': 4 if severity == 'HIGH' else 3 if severity == 'MEDIUM' else 2,
                        'affected_component': 'HTTP headers',
                        'recommendation': f'Implement {header} security header'
                    })

            # Check for insecure headers
            if 'server' in headers and headers['server']:
                findings.append({
                    'type': 'Information Disclosure',
                    'severity': 'LOW',
                    'details': f"Server header: {headers['server']}",
                    'url': url,
                    'timestamp': datetime.now().isoformat(),
                    'evidence': f"Server information exposed in headers: {headers['server']}",
                    'risk_score': 2,
                    'affected_component': 'HTTP headers',
                    'recommendation': 'Remove or obfuscate server header'
                })

        except Exception as e:
            logger.warning(f"Header check failed for {url}: {e}")
        return findings

    def check_miner(self, url: str) -> List[Dict]:
        findings = []
        try:
            resp = self.session.get(url)
            for pattern in self.miner_patterns:
                if pattern.search(resp.text):
                    match = pattern.search(resp.text)
                    evidence_location = self._find_evidence_location(resp.text, match.group())
                    findings.append({
                        'type': 'Cryptominer',
                        'severity': 'HIGH',
                        'details': 'Miner script detected',
                        'url': url,
                        'timestamp': datetime.now().isoformat(),
                        'evidence': f"Cryptomining script pattern detected: {pattern.pattern}. Location: {evidence_location}",
                        'risk_score': 7,
                        'affected_component': 'Web page content',
                        'recommendation': 'Remove cryptomining scripts and scan for malware'
                    })
                    break
        except Exception as e:
            logger.warning(f"Miner check failed for {url}: {e}")
        return findings

    def check_open_redirect(self, url: str) -> List[Dict]:
        findings = []
        for payload in self.payloads['redirect']:
            test_url = f"{url}{'&' if '?' in url else '?'}redirect={quote(payload)}"
            try:
                resp = self.session.get(test_url, allow_redirects=False)
                if resp.status_code in [301, 302, 303, 307, 308]:
                    location = resp.headers.get('location', '')
                    if payload in location or 'evil.com' in location:
                        findings.append({
                            'type': 'Open Redirect',
                            'severity': 'MEDIUM',
                            'details': f"Redirect to: {location}",
                            'url': test_url,
                            'timestamp': datetime.now().isoformat(),
                            'evidence': f"Redirect to external domain detected in Location header: {location}",
                            'risk_score': 5,
                            'affected_component': 'Redirect mechanism',
                            'recommendation': 'Validate redirect URLs and use allowlists'
                        })
            except Exception as e:
                logger.warning(f"Redirect check failed for {test_url}: {e}")
        return findings

    def check_idor(self, url: str) -> List[Dict]:
        findings = []
        for payload in self.payloads['idor']:
            test_url = urljoin(url, payload)
            try:
                resp = self.session.get(test_url)
                if resp.status_code == 200 and resp.url == test_url:
                    findings.append({
                        'type': 'IDOR',
                        'severity': 'HIGH',
                        'details': f"Possible IDOR: {payload}",
                        'url': test_url,
                        'timestamp': datetime.now().isoformat(),
                        'evidence': f"Direct object reference accessible without proper authorization: {test_url}",
                        'risk_score': 7,
                        'affected_component': 'Access control',
                        'recommendation': 'Implement proper access control checks'
                    })
            except Exception as e:
                logger.warning(f"IDOR check failed for {test_url}: {e}")
        return findings

    def check_csrf(self, url: str) -> List[Dict]:
        findings = []
        try:
            resp = self.session.get(url)
            if self.csrf_pattern.search(resp.text):
                findings.append({
                    'type': 'CSRF Protection',
                    'severity': 'INFO',
                    'details': 'CSRF token found (good practice)',
                    'url': url,
                    'timestamp': datetime.now().isoformat(),
                    'evidence': 'CSRF token detected in form input',
                    'risk_score': 0,
                    'affected_component': 'Form security',
                    'recommendation': 'Continue using CSRF protection'
                })
            else:
                # Check forms without CSRF protection
                if BS4_AVAILABLE:
                    soup = BeautifulSoup(resp.text, 'html.parser')
                    forms = soup.find_all('form')
                    for form in forms:
                        if form.find('input', {'name': re.compile(r'csrf|token', re.I)}) is None:
                            findings.append({
                                'type': 'CSRF',
                                'severity': 'MEDIUM',
                                'details': 'Form without CSRF protection',
                                'url': url,
                                'timestamp': datetime.now().isoformat(),
                                'evidence': 'Form found without CSRF token in form inputs',
                                'risk_score': 6,
                                'affected_component': 'Form security',
                                'recommendation': 'Implement CSRF tokens for all forms'
                            })
                            break
        except Exception as e:
            logger.warning(f"CSRF check failed for {url}: {e}")
        return findings

    def _find_evidence_location(self, text: str, pattern: str) -> str:
        """Find the approximate location of evidence in the text."""
        try:
            position = text.find(pattern)
            if position == -1:
                return "Unknown location"

            # Calculate line number
            line_number = text[:position].count('\n') + 1

            # Get context around the finding
            start = max(0, position - 50)
            end = min(len(text), position + len(pattern) + 50)
            context = text[start:end].replace('\n', ' ').replace('\r', ' ')

            return f"Line ~{line_number}, Context: '{context}'"
        except:
            return "Unknown location"

    def full_check(self, url: str) -> Dict:
        print_colored(f"🔍 Checking {url}...", Fore.BLUE)
        start_time = time.time()
        all_findings = []

        # Get site information first
        site_info = self.get_site_info(url)

        # Run all security checks
        all_findings.extend(self.check_xss(url))
        all_findings.extend(self.check_sqli(url))
        all_findings.extend(self.check_lfi(url))
        all_findings.extend(self.check_headers(url))
        all_findings.extend(self.check_miner(url))
        all_findings.extend(self.check_open_redirect(url))
        all_findings.extend(self.check_idor(url))
        all_findings.extend(self.check_csrf(url))

        elapsed_time = time.time() - start_time

        return {
            'url': url,
            'timestamp': datetime.now().isoformat(),
            'findings': all_findings,
            'total_findings': len(all_findings),
            'scan_time': elapsed_time,
            'site_info': site_info
        }

def simple_crawl(base_url: str, session, max_pages=MAX_PAGES):
    """Lightweight crawler that also maps site structure."""
    if not BS4_AVAILABLE:
        return [base_url]

    urls = [base_url]
    visited = {base_url}
    queue = [(base_url, 0)]
    site_structure = {}

    while queue and len(urls) < max_pages:
        current, depth = queue.pop(0)
        if depth >= 2:  # Limit depth for performance
            continue

        try:
            resp = session.get(current)
            soup = BeautifulSoup(resp.text, 'html.parser')

            # Extract page structure information
            page_links = []
            for link in soup.find_all('a', href=True)[:5]:  # Limit links per page
                href = link['href']
                if href.startswith('javascript:') or href.startswith('mailto:'):
                    continue

                next_url = urljoin(current, href)
                parsed_url = urlparse(next_url)

                # Only follow same-domain links
                if parsed_url.netloc == urlparse(base_url).netloc and next_url not in visited:
                    visited.add(next_url)
                    queue.append((next_url, depth + 1))
                    urls.append(next_url)
                    page_links.append(next_url)

                    if len(urls) >= max_pages:
                        break

            # Store page structure
            site_structure[current] = {
                'depth': depth,
                'links': page_links,
                'title': soup.title.string if soup.title else 'No title',
                'forms': len(soup.find_all('form'))
            }

        except Exception as e:
            logger.warning(f"Crawling failed for {current}: {e}")

    return urls, site_structure

In [54]:
class Report:
    """Enhanced report generator with DataFrame support."""
    def __init__(self, results: List[Dict]):
        self.results = results
        self.df = self._create_dataframe()
        self.summary_df = self._create_summary_dataframe()

    def _create_dataframe(self):
        """Create a pandas DataFrame from scan results."""
        data = []
        for res in self.results:
            for finding in res['findings']:
                data.append({
                    'S.No': len(data) + 1,
                    'URL': finding.get('url', res['url']),
                    'Type': finding['type'],
                    'Severity': finding['severity'],
                    'Details': finding['details'],
                    'Timestamp': finding.get('timestamp', ''),
                    'Risk Score': finding.get('risk_score', 0),
                    'Affected Component': finding.get('affected_component', 'Unknown'),
                    'Evidence': finding.get('evidence', ''),
                    'Recommendation': finding.get('recommendation', '')
                })

        return pd.DataFrame(data) if data else pd.DataFrame(columns=['S.No', 'URL', 'Type', 'Severity', 'Details', 'Timestamp', 'Risk Score', 'Affected Component', 'Evidence', 'Recommendation'])

    def _create_summary_dataframe(self):
        """Create a summary DataFrame."""
        if not self.results:
            return pd.DataFrame()

        # Site information
        site_info = self.results[0]['site_info']

        # Severity counts
        severity_counts = self.df['Severity'].value_counts().to_dict() if not self.df.empty else {}

        # Overall risk score
        overall_risk = self.df['Risk Score'].mean() if not self.df.empty else 0

        # Create summary data
        summary_data = {
            'Scan Date': [datetime.now().strftime('%Y-%m-%d %H:%M:%S')],
            'Target URL': [site_info.get('url', 'Unknown')],
            'Domain': [site_info.get('domain', 'Unknown')],
            'IP Address': [site_info.get('ip_address', 'Unknown')],
            'Server': [site_info.get('server', 'Unknown')],
            'Technologies': [', '.join(site_info.get('technologies', []))],
            'Total Pages Scanned': [len(self.results)],
            'Total Findings': [len(self.df)],
            'Critical Findings': [severity_counts.get('CRITICAL', 0)],
            'High Findings': [severity_counts.get('HIGH', 0)],
            'Medium Findings': [severity_counts.get('MEDIUM', 0)],
            'Low Findings': [severity_counts.get('LOW', 0)],
            'Info Findings': [severity_counts.get('INFO', 0)],
            'Overall Risk Score': [f"{overall_risk:.2f}/10"]
        }

        return pd.DataFrame(summary_data)

    def generate(self, format_type='html', export=False):
        """Generate report in various formats."""
        if self.df.empty:
            print_colored("No findings to report.", Fore.YELLOW)
            return None

        # Count findings by severity
        severity_counts = self.df['Severity'].value_counts().to_dict()

        # Calculate overall risk score
        overall_risk = self.df['Risk Score'].mean() if not self.df.empty else 0

        if format_type == 'html':
            report_content = self._generate_html_report(severity_counts, overall_risk)
        elif format_type == 'markdown':
            report_content = self._generate_markdown_report(severity_counts, overall_risk)
        else:
            report_content = self._generate_text_report(severity_counts, overall_risk)

        # Save report
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        report_path = f"{REPORTS_DIR}/duskprobe_report_{timestamp}.{format_type}"

        with open(report_path, 'w', encoding='utf-8') as f:
            f.write(report_content)

        print_colored(f"📄 Report saved: {report_path}", Fore.GREEN)

        # Display summary DataFrame
        if not self.summary_df.empty:
            print_colored("\n📊 SCAN SUMMARY:", Fore.CYAN)
            display(self.summary_df)

        # Display findings DataFrame if not too large
        if len(self.df) <= 50:  # Only display if not too many findings
            print_colored("\n📋 DETAILED FINDINGS:", Fore.CYAN)
            display(self.df)
        else:
            print_colored(f"\n📋 {len(self.df)} findings detected (too many to display)", Fore.CYAN)

        # Export DataFrame if requested
        if export:
            csv_path = f"{REPORTS_DIR}/duskprobe_data_{timestamp}.csv"
            self.df.to_csv(csv_path, index=False)
            print_colored(f"📊 Data exported: {csv_path}", Fore.GREEN)

            # In Colab, provide download link
            if IS_COLAB:
                from google.colab import files
                files.download(csv_path)

        # Display in Colab
        if IS_COLAB:
            if format_type == 'html':
                display(HTML(report_content))
            elif format_type == 'markdown':
                display(Markdown(report_content))

        return report_path

    def _generate_html_report(self, severity_counts, overall_risk):
        """Generate HTML report."""
        # Get site information
        site_info = self.results[0]['site_info'] if self.results else {}

        html = f"""
        <!DOCTYPE html>
        <html>
        <head>
            <title>DuskProbe Security Report</title>
            <style>
                body {{ font-family: Arial, sans-serif; margin: 40px; }}
                h1, h2, h3 {{ color: #2c3e50; }}
                .critical {{ color: #e74c3c; font-weight: bold; }}
                .high {{ color: #e67e22; font-weight: bold; }}
                .medium {{ color: #f39c12; }}
                .low {{ color: #3498db; }}
                .info {{ color: #2ecc71; }}
                table {{ border-collapse: collapse; width: 100%; margin-top: 20px; margin-bottom: 20px; }}
                th, td {{ border: 1px solid #ddd; padding: 8px; text-align: left; }}
                th {{ background-color: #f2f2f2; }}
                tr:nth-child(even) {{ background-color: #f9f9f9; }}
                a {{ color: #3498db; text-decoration: none; }}
                a:hover {{ text-decoration: underline; }}
                .risk-meter {{
                    width: 100%;
                    background-color: #f0f0f0;
                    border-radius: 5px;
                    margin: 10px 0;
                    overflow: hidden;
                }}
                .risk-fill {{
                    height: 20px;
                    background: linear-gradient(to right, #2ecc71, #f39c12, #e74c3c);
                    width: {overall_risk * 10}%;
                    border-radius: 5px;
                }}
                .site-info {{ background-color: #f8f9fa; padding: 15px; border-radius: 5px; margin-bottom: 20px; }}
                .summary-table {{ width: 100%; border-collapse: collapse; }}
                .summary-table th, .summary-table td {{ padding: 8px; text-align: left; border-bottom: 1px solid #ddd; }}
            </style>
        </head>
        <body>
            <h1>🛡️ DuskProbe Security Report</h1>
            <p><strong>Generated:</strong> {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}</p>

            <div class="site-info">
                <h2>Site Information</h2>
                <table class="summary-table">
                    <tr><th>Target URL</th><td>{site_info.get('url', 'Unknown')}</td></tr>
                    <tr><th>Domain</th><td>{site_info.get('domain', 'Unknown')}</td></tr>
                    <tr><th>IP Address</th><td>{site_info.get('ip_address', 'Unknown')}</td></tr>
                    <tr><th>Server</th><td>{site_info.get('server', 'Unknown')}</td></tr>
                    <tr><th>Technologies</th><td>{', '.join(site_info.get('technologies', []))}</td></tr>
                    <tr><th>Page Title</th><td>{site_info.get('page_title', 'Unknown')}</td></tr>
                </table>
            </div>

            <h2>Executive Summary</h2>
            <div class="risk-meter">
                <div class="risk-fill"></div>
            </div>
            <p><strong>Overall Risk Score:</strong> {overall_risk:.2f}/10</p>

            <h3>Findings Summary</h3>
            <table class="summary-table">
                <tr><th>Critical</th><td class="critical">{severity_counts.get('CRITICAL', 0)}</td></tr>
                <tr><th>High</th><td class="high">{severity_counts.get('HIGH', 0)}</td></tr>
                <tr><th>Medium</th><td class="medium">{severity_counts.get('MEDIUM', 0)}</td></tr>
                <tr><th>Low</th><td class="low">{severity_counts.get('LOW', 0)}</td></tr>
                <tr><th>Info</th><td class="info">{severity_counts.get('INFO', 0)}</td></tr>
                <tr><th>Total Findings</th><td><strong>{len(self.df)}</strong></td></tr>
            </table>

            <h2>Detailed Findings</h2>
            <table>
                <tr>
                    <th>S.No</th>
                    <th>URL</th>
                    <th>Type</th>
                    <th>Severity</th>
                    <th>Details</th>
                    <th>Risk Score</th>
                    <th>Affected Component</th>
                    <th>Evidence</th>
                    <th>Recommendation</th>
                </tr>
        """

        for _, row in self.df.iterrows():
            severity_class = row['Severity'].lower()
            html += f"""
                <tr>
                    <td>{row['S.No']}</td>
                    <td><a href="{row['URL']}" target="_blank">{row['URL']}</a></td>
                    <td>{row['Type']}</td>
                    <td class="{severity_class}">{row['Severity']}</td>
                    <td>{row['Details']}</td>
                    <td>{row['Risk Score']}</td>
                    <td>{row['Affected Component']}</td>
                    <td>{row['Evidence']}</td>
                    <td>{row['Recommendation']}</td>
                </tr>
            """

        html += """
            </table>
        </body>
        </html>
        """

        return html

    def _generate_markdown_report(self, severity_counts, overall_risk):
        """Generate Markdown report."""
        # Get site information
        site_info = self.results[0]['site_info'] if self.results else {}

        md = f"""# DuskProbe Security Report

**Generated:** {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}

## Site Information

| Attribute | Value |
|-----------|-------|
| Target URL | {site_info.get('url', 'Unknown')} |
| Domain | {site_info.get('domain', 'Unknown')} |
| IP Address | {site_info.get('ip_address', 'Unknown')} |
| Server | {site_info.get('server', 'Unknown')} |
| Technologies | {', '.join(site_info.get('technologies', []))} |
| Page Title | {site_info.get('page_title', 'Unknown')} |

## Executive Summary

**Overall Risk Score:** {overall_risk:.2f}/10

### Findings Summary

| Severity | Count |
|----------|-------|
| Critical | {severity_counts.get('CRITICAL', 0)} |
| High | {severity_counts.get('HIGH', 0)} |
| Medium | {severity_counts.get('MEDIUM', 0)} |
| Low | {severity_counts.get('LOW', 0)} |
| Info | {severity_counts.get('INFO', 0)} |
| **Total** | **{len(self.df)}** |

## Detailed Findings

| S.No | URL | Type | Severity | Details | Risk Score | Affected Component | Evidence | Recommendation |
|------|-----|------|----------|---------|------------|-------------------|----------|----------------|
"""

        for _, row in self.df.iterrows():
            md += f"| {row['S.No']} | [{row['URL']}]({row['URL']}) | {row['Type']} | {row['Severity']} | {row['Details']} | {row['Risk Score']} | {row['Affected Component']} | {row['Evidence']} | {row['Recommendation']} |\n"

        return md

    def _generate_text_report(self, severity_counts, overall_risk):
        """Generate text report."""
        # Get site information
        site_info = self.results[0]['site_info'] if self.results else {}

        text = f"DUSKPROBE SECURITY REPORT\n"
        text += f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n"

        text += "SITE INFORMATION:\n"
        text += f"  Target URL: {site_info.get('url', 'Unknown')}\n"
        text += f"  Domain: {site_info.get('domain', 'Unknown')}\n"
        text += f"  IP Address: {site_info.get('ip_address', 'Unknown')}\n"
        text += f"  Server: {site_info.get('server', 'Unknown')}\n"
        text += f"  Technologies: {', '.join(site_info.get('technologies', []))}\n"
        text += f"  Page Title: {site_info.get('page_title', 'Unknown')}\n\n"

        text += "EXECUTIVE SUMMARY:\n"
        text += f"  Overall Risk Score: {overall_risk:.2f}/10\n\n"

        text += "FINDINGS SUMMARY:\n"
        text += f"  Critical: {severity_counts.get('CRITICAL', 0)}\n"
        text += f"  High: {severity_counts.get('HIGH', 0)}\n"
        text += f"  Medium: {severity_counts.get('MEDIUM', 0)}\n"
        text += f"  Low: {severity_counts.get('LOW', 0)}\n"
        text += f"  Info: {severity_counts.get('INFO', 0)}\n"
        text += f"  Total Findings: {len(self.df)}\n\n"

        text += "DETAILED FINDINGS:\n"
        for _, row in self.df.iterrows():
            text += f"  S.No: {row['S.No']}\n"
            text += f"  URL: {row['URL']}\n"
            text += f"  Type: {row['Type']}\n"
            text += f"  Severity: {row['Severity']}\n"
            text += f"  Details: {row['Details']}\n"
            text += f"  Risk Score: {row['Risk Score']}\n"
            text += f"  Affected Component: {row['Affected Component']}\n"
            text += f"  Evidence: {row['Evidence']}\n"
            text += f"  Recommendation: {row['Recommendation']}\n"
            text += "-" * 80 + "\n"

        return text

In [55]:
# =============================================================================
# INTERACTIVE MAIN FUNCTION - ASKS FOR URL INPUT
# =============================================================================
def run_duskprobe():
    """Interactive security check - asks for URL."""
    print_colored("\n" + "="*60, Fore.MAGENTA)
    print_colored("🛡️  DUSKPROBE v4.5 - ENHANCED SECURITY SCANNER", Fore.MAGENTA)
    print_colored("="*60, Fore.MAGENTA)

    # Ethical disclaimer
    print_colored("\n⚠️  ETHICAL AND LEGAL DISCLAIMER:", Fore.YELLOW)
    print_colored("This tool is for educational and authorized security testing only.", Fore.YELLOW)
    print_colored("Unauthorized use against systems you don't own is illegal.", Fore.YELLOW)

    # User confirmation
    age_confirmation = input("Are you 18 years or older? (y/n): ").strip().lower()
    if age_confirmation != 'y':
        print_colored("❌ You must be 18+ to use this tool.", Fore.RED)
        return

    ethical_confirmation = input("Do you have permission to scan the target? (y/n): ").strip().lower()
    if ethical_confirmation != 'y':
        print_colored("❌ You must have proper authorization to scan targets.", Fore.RED)
        return

    # Ask for URL
    url = input("Enter URL to scan (e.g., https://example.com): ").strip()
    if not url or not url.startswith(('http://', 'https://')):
        print_colored("❌ Invalid URL! Must start with http:// or https://", Fore.RED)
        return

    use_tor = input("Use Tor for anonymity? (y/n): ").strip().lower() == 'y'
    crawl = input("Enable crawling (slower but thorough)? (y/n): ").strip().lower() == 'y'
    export_data = input("Export data to CSV? (y/n): ").strip().lower() == 'y'

    print_colored(f"\n🚀 Starting security check on {url} {'with Tor' if use_tor else ''}...", Fore.BLUE)

    session = SimpleSession(use_tor)
    checker = SecurityChecker(session)

    try:
        start_time = time.time()

        # Initial check
        result = checker.full_check(url)
        results = [result]

        # Crawl if enabled
        site_structure = {}
        if crawl and BS4_AVAILABLE:
            print_colored("🌐 Crawling site and mapping structure...", Fore.YELLOW)
            sub_urls, site_structure = simple_crawl(url, session)
            print_colored(f"Found {len(sub_urls)} pages to scan", Fore.YELLOW)

            for i, sub_url in enumerate(sub_urls[1:], 1):
                print_colored(f"Scanning page {i}/{len(sub_urls)-1}: {sub_url}", Fore.BLUE)
                sub_result = checker.full_check(sub_url)
                results.append(sub_result)

        # Generate report
        report = Report(results)
        report.generate('html', export_data)

        # Display site structure if available
        if site_structure:
            print_colored("\n🏗️  SITE STRUCTURE:", Fore.CYAN)
            structure_data = []
            for page, info in site_structure.items():
                structure_data.append({
                    'URL': page,
                    'Depth': info['depth'],
                    'Title': info['title'],
                    'Links Found': len(info['links']),
                    'Forms': info['forms']
                })
            structure_df = pd.DataFrame(structure_data)
            display(structure_df)

        # Summary
        total_time = time.time() - start_time
        total_critical = sum(1 for r in results for f in r['findings'] if f['severity'] == 'CRITICAL')
        total_high = sum(1 for r in results for f in r['findings'] if f['severity'] == 'HIGH')
        total_findings = sum(len(r['findings']) for r in results)

        print_colored(f"\n✅ SCAN COMPLETE!", Fore.GREEN)
        print_colored(f"   Scan time: {total_time:.2f} seconds", Fore.WHITE)
        print_colored(f"   Critical: {total_critical} | High: {total_high} | Total: {total_findings}", Fore.CYAN)
        print_colored(f"   Log file: {log_file}", Fore.WHITE)

        if total_critical > 0:
            print_colored("⚠️  CRITICAL VULNERABILITIES FOUND - TAKE ACTION IMMEDIATELY!", Fore.RED)
        elif total_high > 0:
            print_colored("⚠️  HIGH-RISK ISSUES DETECTED - REVIEW URGENTLY", Fore.YELLOW)
        else:
            print_colored("✅ No major issues found - but always verify manually!", Fore.GREEN)

    except Exception as e:
        print_colored(f"❌ Scan failed: {e}", Fore.RED)
        logger.error(f"Scan failed: {e}")
    finally:
        session.close()
        print_colored("\nDuskProbe. Developed by : Labib Bin Shahed", Fore.MAGENTA)


In [56]:
# Run the interactive scanner
if __name__ == "__main__":
    run_duskprobe()

2025-09-09 14:37:36,051 - DuskProbe - INFO - 
INFO:DuskProbe:


🛡️  DUSKPROBE v4.5 - ENHANCED SECURITY SCANNER


2025-09-09 14:37:36,061 - DuskProbe - INFO - 🛡️  DUSKPROBE v4.5 - ENHANCED SECURITY SCANNER
INFO:DuskProbe:🛡️  DUSKPROBE v4.5 - ENHANCED SECURITY SCANNER


2025-09-09 14:37:36,070 - DuskProbe - INFO - ============================================================
INFO:DuskProbe:============================================================



⚠️  ETHICAL AND LEGAL DISCLAIMER:


...

2025-09-09 14:38:12,354 - DuskProbe - INFO -    Log file: /content/logs/duskprobe_20250909_143735.log
INFO:DuskProbe:   Log file: /content/logs/duskprobe_20250909_143735.log


⚠️  CRITICAL VULNERABILITIES FOUND - TAKE ACTION IMMEDIATELY!


2025-09-09 14:38:12,362 - DuskProbe - INFO - ⚠️  CRITICAL VULNERABILITIES FOUND - TAKE ACTION IMMEDIATELY!
INFO:DuskProbe:⚠️  CRITICAL VULNERABILITIES FOUND - TAKE ACTION IMMEDIATELY!



DuskProbe. Developed by : Labib Bin Shahed


2025-09-09 14:38:12,370 - DuskProbe - INFO - 
DuskProbe. Developed by : Labib Bin Shahed
INFO:DuskProbe:
DuskProbe. Developed by : Labib Bin Shahed
